<img src = "https://images2.imgbox.com/32/ac/wucGkuem_o.png" width="300">

In [ ]:
#!python -m spacy download pt_core_news_sm

In [6]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors

w2v_modelo_cbow = KeyedVectors.load_word2vec_format("models/modelo_cbow.txt")
w2v_modelo_sg = KeyedVectors.load_word2vec_format("models/modelo_skipgram.txt")
artigo_treino = pd.read_csv("treino.csv")
artigo_teste = pd.read_csv("teste.csv")

In [7]:
nlp = spacy.load("pt_core_news_sm")

def tokenizador(texto):
    doc = nlp(texto)
    tokens_validos = []
    
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())

    
    return  tokens_validos

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
tokens_exemplo = tokenizador(texto)
print(tokens_exemplo)

['rio', 'janeiro', 'cidade', 'maravilhosa']


In [8]:
def combinacao_de_vetores_por_soma(palavras, modelo):
    
    vetor_resultante = np.zeros((1,300))
    
    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)

        except KeyError:
            pass
    
    return vetor_resultante

vetor_texto = combinacao_de_vetores_por_soma(tokens_exemplo, w2v_modelo_cbow)

In [9]:
vetor_texto.shape

(1, 300)

In [10]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz

matriz_vetores_treino = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)
matriz_vetores_teste = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)
print(matriz_vetores_treino.shape)
print(matriz_vetores_teste.shape) 

(90000, 300)
(20513, 300)


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

LR_cbow = LogisticRegression(max_iter = 800)
LR_cbow.fit(matriz_vetores_treino, artigo_treino.category)
categorias_previsao = LR_cbow.predict(matriz_vetores_teste)
resultado = classification_report(artigo_teste.category, categorias_previsao)
print(resultado)

              precision    recall  f1-score   support

     colunas       0.80      0.71      0.75      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.13      0.83      0.22       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.83      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.80      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [12]:
matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)
print(matriz_vetores_treino_sg.shape)
print(matriz_vetores_teste_sg.shape) 

(90000, 300)
(20513, 300)


In [13]:
LR_sg = LogisticRegression(max_iter = 800)
LR_sg.fit(matriz_vetores_treino_sg, artigo_treino.category)
categorias_previsao_sg = LR_sg.predict(matriz_vetores_teste_sg)
resultado_sg = classification_report(artigo_teste.category, categorias_previsao_sg)
print(resultado_sg)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.64      0.81      0.71      1698
     esporte       0.93      0.87      0.90      4663
   ilustrada       0.14      0.87      0.24       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.75      0.85      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.69      0.82      0.71     20513
weighted avg       0.82      0.79      0.80     20513



In [14]:
import pickle

with open("models/rl_cbow.pkl", "wb") as f:
    pickle.dump(LR_cbow, f)

In [15]:
with open("models/rl_sg.pkl", "wb") as f:
    pickle.dump(LR_sg, f)